In this demo, we try to see how using different embeddings might result in different optimized architectures whilst keeping other hyper parameters and data the same. 

We have six built-in embeddings available. They are represented by the string 'E1' to 'E6' as:
    
    'E1': angle embedding with Rot_Y
    'E2': amplitude embedding
    'E3': qaoa variational hamiltonian ansatz inspired embedding
    'E4': XXZ variational hamiltonian ansatz inspired embedding
    'E5': the most expressive circuit from aspuru guzik group 2019 paper
    'E6': random embedding with randomly selected rot gates and CNOTs
        
As with other system specifications, you can set the embedding you want to use in the system configuration dictionary.

In [1]:
#First we import the essentials

In [8]:
import pennylane as qml
from pennylane import numpy as np
import networkx as nx
import os
import time
import pickle

In [3]:
from qose.subarchitecture_tree_search import run_tree_architecture_search

In [4]:
    # Create a unique name for your experiment
    EXPERIMENT_NAME = 'VariousEmbeddings'

    # Create a directory to store the data
    if not os.path.exists('data'):
        os.mkdir('data/')

    data_path = f'data/{EXPERIMENT_NAME}'
    if not os.path.exists(data_path):
        os.mkdir(data_path)

In [5]:
config = {'nqubits': 2,  # number of qubits in the circuit
          'embedding': 'E1', # what embedding to use
          'min_tree_depth': 4,  # minimum depth before we start pruning
          'max_tree_depth': 6,  # maximum circuit depth
          'prune_rate': 0.15, # rate with which to prune leaves
          'prune_step': 3, # prune only if we have reached min_tree_depth and if (depth % prune_step) == 0
          'plot_trees': False, # whether or not to plot trees during algorithm (graphviz required!)
          'data_set': 'moons',  # the data set
          'nsteps': 20, # number of steps for variational training
          'optim': qml.AdamOptimizer, # pennylane optimizer
          'batch_sizes': [8], # batch sizes to check for each circuit we construct.
          'n_samples': 1500, # number of samples of the data
          'learning_rates': [0.01], # learning rates to check for each circuit we construct.
          'save_frequency': 1, # how often save the tree to pickle, 1 means always, 0 never
          'save_path': data_path, # where to save the data
          'nprocesses': 5, # number of MPI processes spawned, controls the degree of parallelization.
          'save_timing': False, # store the time how long each circuit tranining takes
          'circuit_type': 'schuld', # specify the list of possible layers (see docs for more info)
          'Tmax': [100, 100, 100], # parameters for w-cost function
          'inf_time': 'numcnots', # how to determine circuit time, can be with CNOT number of clock time.
          'fill': 'redundant',  # embedding parameter 
          'rate_type': 'accuracy',  # how to evaluate error rate for w-cost
          'readout_layer': 'one_hot',  # how to do classiciation.
         }

In [6]:
possible_emb = ['E1', 'E2', 'E3','E4','E5','E6']
for key in possible_emb:
    config['embedding'] = key
    print("##########Embedding: ", key)
    t_0_local = time.time()
    run_tree_architecture_search(config, "local")
    t_1_local = time.time()
    print("*********Time taken for",key," embedding: ", t_1_local - t_0_local )


##########Embedding:  E1
Depth = 1
Depth = 2
Current best architecture:  E1
max W: 1.0
weights: []
Training leaf E1:ZZ
Training leaf E1:X
Training leaf E1:Y
Training leaf E1:Z
Depth = 3
Current best architecture:  E1:X
max W: 2.070422097798148
weights: [[0.18432993]
 [0.11447192]]
Training leaf E1:ZZ:X
Training leaf E1:ZZ:Y
Training leaf E1:ZZ:Z
Training leaf E1:X:ZZ
Training leaf E1:X:Y
Training leaf E1:X:Z
Training leaf E1:Y:ZZ
Training leaf E1:Y:X
Training leaf E1:Y:Z
Training leaf E1:Z:ZZ
Training leaf E1:Z:X
Training leaf E1:Z:Y
Depth = 4
Prune Tree
Current best architecture:  E1:Y:X
max W: 2.2615379396450908
weights: [[0.09355039 0.09516496]
 [0.1702017  0.17383168]]
Grow Pruned Tree
Depth = 5
Grow Tree
Current best architecture:  E1:X:Y:X
max W: 2.449999387500153
weights: [[0.14993716 0.17651175 0.14987826]
 [0.18447972 0.17387657 0.18443745]]
architecture with max W:  E1:Y:Z:Y:X
max W: 2.999999081632934
weights:  [[ 0.12262601 -0.17781427  0.12071927  0.12554879]
 [ 0.16658778 

In [9]:
with open(data_path + '/config.pickle', 'wb') as f:
        pickle.dump(config, f)